# GEE IO

## Setup

### Ecoscope

In [ ]:
%pip install \
    'ecoscope[analysis,mapping,plotting] @ git+https://github.com/wildlife-dynamics/ecoscope@v1.8.36' &> /dev/null

In [ ]:
import os
import sys

import shapely

import ecoscope
from ecoscope.contrib import geemap

ecoscope.init()

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

### Earth Engine

In [ ]:
import ee

try:
    EE_ACCOUNT = os.getenv("EE_ACCOUNT")
    EE_PRIVATE_KEY_DATA = os.getenv("EE_PRIVATE_KEY_DATA")
    if EE_ACCOUNT and EE_PRIVATE_KEY_DATA:
        geemap.ee_initialize(credentials=ee.ServiceAccountCredentials(EE_ACCOUNT, key_data=EE_PRIVATE_KEY_DATA))
    else:
        geemap.ee_initialize()

except ee.EEException:
    ee.Authenticate()
    geemap.ee_initialize()

## Define Region as an EarthEngine Geometry

In [ ]:
region_geo = ee.Feature(shapely.geometry.mapping(shapely.geometry.Point(5.178332, -1.200788).buffer(0.3))).geometry()

## Define the Image to Sample

In [ ]:
img = ee.Image("USGS/SRTMGL1_003")

## Define the Download Configuration

Options are: description, folder, fileNamePrefix, dimensions, region, scale, crs, crsTransform, maxPixels, shardSize, fileDimensions, skipEmptyTiles, fileFormat, formatOptions

In [ ]:
download_config = {
    "description": "dem",
    "folder": "Ecoscope-Outputs",
    "fileNamePrefix": "dem",
    "region": region_geo.getInfo()["coordinates"],
    "scale": 30,
    "fileFormat": "geotiff",
}

## Method 1: Download Using a Google Drive Background Task

In [ ]:
task = ee.batch.Export.image.toDrive(img, **download_config)
task.start()

### Check Task Status

In [ ]:
task.status().get("state")

## Method 2: Direct Download (Size-Limited)

In [ ]:
img_zip_file = os.path.join(output_dir, "img_file.zip")

ecoscope.io.utils.download_file(
    url=img.getDownloadUrl(download_config),
    path=img_zip_file,
    unzip=True,
)